In [42]:
# read data from CSV

import pandas as pd

data = pd.read_csv('f1_data.csv')

data.head()


,driver_number,broadcast_name,country_code_x,year,session_name,date_start,time_start,date_end,time_end,gmt_offset,starting_position,wins_before,pit_stops_count,avg_pit_stop_duration,qualifying_position,position_category
0,1,M VERSTAPPEN,NED,2023,Race,2023-03-19,17:00:00,2023-03-19,19:00:00,03:00:00,15,1,0,0.0,1,top3
1,2,L SARGEANT,USA,2023,Race,2023-03-19,17:00:00,2023-03-19,19:00:00,03:00:00,20,0,0,0.0,16,no_points
2,4,L NORRIS,GBR,2023,Race,2023-03-19,17:00:00,2023-03-19,19:00:00,03:00:00,19,0,0,0.0,7,no_points
3,10,P GASLY,FRA,2023,Race,2023-03-19,17:00:00,2023-03-19,19:00:00,03:00:00,9,0,0,0.0,9,points
4,11,S PEREZ,MEX,2023,Race,2023-03-19,17:00:00,2023-03-19,19:00:00,03:00:00,1,0,0,0.0,2,winner


In [43]:
# data without position category 
X = data.drop(columns=['position_category'])
y = data['position_category']



In [44]:
# wyswitel informacje o zbiorze danych
print("Shape of X:", X.shape)
print("Shape of y:", y.shape)

#wyswietl cechy
print("Features in X:", X.columns.tolist())
# wyswietl unikalne kategorie pozycji
print("Unique position categories in y:", y.unique())

Shape of X: (1635, 15)
Shape of y: (1635,)
Features in X: ['driver_number', 'broadcast_name', 'country_code_x', 'year', 'session_name', 'date_start', 'time_start', 'date_end', 'time_end', 'gmt_offset', 'starting_position', 'wins_before', 'pit_stops_count', 'avg_pit_stop_duration', 'qualifying_position']
Unique position categories in y: ['top3' 'no_points' 'points' 'winner']


In [45]:
from sklearn.preprocessing import LabelEncoder, StandardScaler

for col in X.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])

In [46]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, log_loss, mean_squared_error


In [47]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [48]:
X_train, X_valid, y_train, y_valid = train_test_split(X_scaled, y, test_size=0.3, stratify=y, random_state=42)

In [49]:
from sklearn.preprocessing import LabelEncoder


In [51]:
def compute_specificity(conf_mat):
    specificity = []
    for i in range(conf_mat.shape[0]):
        TP = conf_mat[i, i]
        FP = conf_mat[:, i].sum() - TP
        FN = conf_mat[i, :].sum() - TP
        TN = conf_mat.sum() - (TP + FP + FN)
        spec = TN / (TN + FP) if (TN + FP) > 0 else 0
        specificity.append(spec)
    return specificity

In [52]:
from sklearn.model_selection import StratifiedKFold
import numpy as np

def objective(trial):
    # === Hiperparametry ===
    rf_n_estimators = trial.suggest_int("rf_n_estimators", 50, 150)
    rf_max_depth = trial.suggest_int("rf_max_depth", 3, 15)

    gb_n_estimators = trial.suggest_int("gb_n_estimators", 50, 150)
    gb_learning_rate = trial.suggest_float("gb_learning_rate", 0.01, 0.3)
    gb_max_depth = trial.suggest_int("gb_max_depth", 2, 10)

    svm_C = trial.suggest_float("svm_C", 0.1, 10.0, log=True)
    svm_kernel = trial.suggest_categorical("svm_kernel", ["linear", "rbf"])

    # === Modele ===
    clf_rf = RandomForestClassifier(n_estimators=rf_n_estimators, max_depth=rf_max_depth, random_state=42)
    clf_gb = GradientBoostingClassifier(n_estimators=gb_n_estimators, learning_rate=gb_learning_rate,
                                        max_depth=gb_max_depth, random_state=42)
    clf_svm = SVC(C=svm_C, kernel=svm_kernel, probability=True, random_state=42)

    # === Ensemble ===
    ensemble = VotingClassifier(
        estimators=[('rf', clf_rf), ('gb', clf_gb), ('svm', clf_svm)],
        voting='soft'
    )

    # === Label encoding ===
    le = LabelEncoder()
    y_train_enc = le.fit_transform(y_train)
    y_valid_enc = le.transform(y_valid)

    class_labels = le.classes_
    print("Klasy w macierzy pomyłek:", class_labels)

    # === k-Fold Cross-Validation ===
    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    acc_list, prec_list, rec_list, f1_list, spec_list = [], [], [], [], []
    mse_list, rmse_list, logloss_list = [], [], []

    # nowe metryki
    prec_macro_list, prec_weighted_list = [], []
    rec_macro_list, rec_weighted_list = [], []
    f1_macro_list, f1_weighted_list = [], []

    # globalna macierz pomyłek
    conf_mat_sum = np.zeros((len(np.unique(y_train_enc)), len(np.unique(y_train_enc))), dtype=int)

    for train_index, test_index in kf.split(X_train, y_train_enc):
        X_tr, X_te = X_train[train_index], X_train[test_index]
        y_tr, y_te = y_train_enc[train_index], y_train_enc[test_index]

        ensemble.fit(X_tr, y_tr)
        y_pred_cv = ensemble.predict(X_te)
        y_proba_cv = ensemble.predict_proba(X_te)

        conf_mat_cv = confusion_matrix(y_te, y_pred_cv)
        conf_mat_sum += conf_mat_cv  # sumowanie po wszystkich foldach

        acc_list.append(accuracy_score(y_te, y_pred_cv))
        prec_list.append(precision_score(y_te, y_pred_cv, average="micro", zero_division=0))
        rec_list.append(recall_score(y_te, y_pred_cv, average="micro", zero_division=0))
        f1_list.append(f1_score(y_te, y_pred_cv, average="micro", zero_division=0))

        # nowe metryki
        prec_macro_list.append(precision_score(y_te, y_pred_cv, average="macro", zero_division=0))
        prec_weighted_list.append(precision_score(y_te, y_pred_cv, average="weighted", zero_division=0))

        rec_macro_list.append(recall_score(y_te, y_pred_cv, average="macro", zero_division=0))
        rec_weighted_list.append(recall_score(y_te, y_pred_cv, average="weighted", zero_division=0))

        f1_macro_list.append(f1_score(y_te, y_pred_cv, average="macro", zero_division=0))
        f1_weighted_list.append(f1_score(y_te, y_pred_cv, average="weighted", zero_division=0))

        spec_cv = np.mean(compute_specificity(conf_mat_cv))
        spec_list.append(spec_cv)

        mse_list.append(mean_squared_error(y_te, y_pred_cv))
        rmse_list.append(np.sqrt(mse_list[-1]))
        logloss_list.append(log_loss(y_te, y_proba_cv))

    # === Średnie i SD dla metryk ===
    acc, acc_std = np.mean(acc_list), np.std(acc_list, ddof=0)
    prec, prec_std = np.mean(prec_list), np.std(prec_list, ddof=0)
    rec, rec_std = np.mean(rec_list), np.std(rec_list, ddof=0)
    f1, f1_std = np.mean(f1_list), np.std(f1_list, ddof=0)
    spec, spec_std = np.mean(spec_list), np.std(spec_list, ddof=0)

    mse, mse_std = np.mean(mse_list), np.std(mse_list, ddof=0)
    rmse, rmse_std = np.mean(rmse_list), np.std(rmse_list, ddof=0)
    logloss, logloss_std = np.mean(logloss_list), np.std(logloss_list, ddof=0)

    # === Średnie i SD dla nowych metryk ===
    prec_macro, prec_macro_std = np.mean(prec_macro_list), np.std(prec_macro_list, ddof=0)
    prec_weighted, prec_weighted_std = np.mean(prec_weighted_list), np.std(prec_weighted_list, ddof=0)

    rec_macro, rec_macro_std = np.mean(rec_macro_list), np.std(rec_macro_list, ddof=0)
    rec_weighted, rec_weighted_std = np.mean(rec_weighted_list), np.std(rec_weighted_list, ddof=0)

    f1_macro, f1_macro_std = np.mean(f1_macro_list), np.std(f1_macro_list, ddof=0)
    f1_weighted, f1_weighted_std = np.mean(f1_weighted_list), np.std(f1_weighted_list, ddof=0)

    # === Zapis do triala ===
    trial.set_user_attr("accuracy", acc)
    trial.set_user_attr("accuracy_std", acc_std)
    trial.set_user_attr("precision", prec)
    trial.set_user_attr("precision_std", prec_std)
    trial.set_user_attr("recall", rec)
    trial.set_user_attr("recall_std", rec_std)
    trial.set_user_attr("f1_score", f1)
    trial.set_user_attr("f1_std", f1_std)
    trial.set_user_attr("specificity_avg", spec)
    trial.set_user_attr("specificity_std", spec_std)

    trial.set_user_attr("mse", mse)
    trial.set_user_attr("mse_std", mse_std)
    trial.set_user_attr("rmse", rmse)
    trial.set_user_attr("rmse_std", rmse_std)
    trial.set_user_attr("logloss", logloss)
    trial.set_user_attr("logloss_std", logloss_std)

    # nowe metryki
    trial.set_user_attr("precision_macro", prec_macro)
    trial.set_user_attr("precision_macro_std", prec_macro_std)
    trial.set_user_attr("precision_weighted", prec_weighted)
    trial.set_user_attr("precision_weighted_std", prec_weighted_std)

    trial.set_user_attr("recall_macro", rec_macro)
    trial.set_user_attr("recall_macro_std", rec_macro_std)
    trial.set_user_attr("recall_weighted", rec_weighted)
    trial.set_user_attr("recall_weighted_std", rec_weighted_std)

    trial.set_user_attr("f1_macro", f1_macro)
    trial.set_user_attr("f1_macro_std", f1_macro_std)
    trial.set_user_attr("f1_weighted", f1_weighted)
    trial.set_user_attr("f1_weighted_std", f1_weighted_std)

    # zapis całej macierzy pomyłek (zsumowanej z 5 foldów)
    trial.set_user_attr("confusion_matrix", conf_mat_sum.tolist())

    return f1  # lub acc jeśli optymalizujesz dokładność


In [53]:
import optuna

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

[I 2025-09-14 01:13:12,636] A new study created in memory with name: no-name-e7c2ba2e-d7f7-4173-9f0a-1f830d48a247


Klasy w macierzy pomyłek: ['no_points' 'points' 'top3' 'winner']


[I 2025-09-14 01:13:22,770] Trial 0 finished with value: 0.8015513675017237 and parameters: {'rf_n_estimators': 100, 'rf_max_depth': 15, 'gb_n_estimators': 78, 'gb_learning_rate': 0.09707853710220406, 'gb_max_depth': 6, 'svm_C': 0.8532952121933164, 'svm_kernel': 'linear'}. Best is trial 0 with value: 0.8015513675017237.


Klasy w macierzy pomyłek: ['no_points' 'points' 'top3' 'winner']


[I 2025-09-14 01:13:42,973] Trial 1 finished with value: 0.8024208994100972 and parameters: {'rf_n_estimators': 58, 'rf_max_depth': 3, 'gb_n_estimators': 144, 'gb_learning_rate': 0.1875845773770531, 'gb_max_depth': 9, 'svm_C': 0.41242558617397584, 'svm_kernel': 'rbf'}. Best is trial 1 with value: 0.8024208994100972.


...

[I 2025-09-14 01:25:37,098] Trial 47 finished with value: 0.8129127403661993 and parameters: {'rf_n_estimators': 124, 'rf_max_depth': 14, 'gb_n_estimators': 82, 'gb_learning_rate': 0.26494711996198134, 'gb_max_depth': 10, 'svm_C': 3.302679285587625, 'svm_kernel': 'rbf'}. Best is trial 21 with value: 0.8207768329119742.


Klasy w macierzy pomyłek: ['no_points' 'points' 'top3' 'winner']


[I 2025-09-14 01:25:53,930] Trial 48 finished with value: 0.815532827702444 and parameters: {'rf_n_estimators': 132, 'rf_max_depth': 14, 'gb_n_estimators': 65, 'gb_learning_rate': 0.24715651197141453, 'gb_max_depth': 10, 'svm_C': 7.583374988108422, 'svm_kernel': 'rbf'}. Best is trial 21 with value: 0.8207768329119742.


Klasy w macierzy pomyłek: ['no_points' 'points' 'top3' 'winner']


[I 2025-09-14 01:26:12,301] Trial 49 finished with value: 0.8225158967287214 and parameters: {'rf_n_estimators': 116, 'rf_max_depth': 15, 'gb_n_estimators': 70, 'gb_learning_rate': 0.21815227101702045, 'gb_max_depth': 10, 'svm_C': 6.512159685772161, 'svm_kernel': 'rbf'}. Best is trial 49 with value: 0.8225158967287214.


In [54]:
best_params = study.best_trial.params

print("🎯 Najlepsze hiperparametry:")
for param, value in best_params.items():
    print(f"{param}: {value}")


🎯 Najlepsze hiperparametry:
rf_n_estimators: 116
rf_max_depth: 15
gb_n_estimators: 70
gb_learning_rate: 0.21815227101702045
gb_max_depth: 10
svm_C: 6.512159685772161
svm_kernel: rbf


In [55]:
best_trial = study.best_trial


print("\n📊 Metryki najlepszej próby:")

# --- Mikro ---
print("--- Mikro ---")
print(f"F1-score (micro): {best_trial.user_attrs['f1_score']:.4f} ± {best_trial.user_attrs['f1_std']:.4f}")
print(f"Accuracy: {best_trial.user_attrs['accuracy']:.4f} ± {best_trial.user_attrs['accuracy_std']:.4f}")
print(f"Precision (micro): {best_trial.user_attrs['precision']:.4f} ± {best_trial.user_attrs['precision_std']:.4f}")
print(f"Recall / Sensitivity (micro): {best_trial.user_attrs['recall']:.4f} ± {best_trial.user_attrs['recall_std']:.4f}")

# --- Makro ---
print("\n--- Makro ---")
print(f"F1-score (macro): {best_trial.user_attrs['f1_macro']:.4f} ± {best_trial.user_attrs['f1_macro_std']:.4f}")
print(f"Precision (macro): {best_trial.user_attrs['precision_macro']:.4f} ± {best_trial.user_attrs['precision_macro_std']:.4f}")
print(f"Recall / Sensitivity (macro): {best_trial.user_attrs['recall_macro']:.4f} ± {best_trial.user_attrs['recall_macro_std']:.4f}")

# --- Weighted ---
print("\n--- Weighted ---")
print(f"F1-score (weighted): {best_trial.user_attrs['f1_weighted']:.4f} ± {best_trial.user_attrs['f1_weighted_std']:.4f}")
print(f"Precision (weighted): {best_trial.user_attrs['precision_weighted']:.4f} ± {best_trial.user_attrs['precision_weighted_std']:.4f}")
print(f"Recall / Sensitivity (weighted): {best_trial.user_attrs['recall_weighted']:.4f} ± {best_trial.user_attrs['recall_weighted_std']:.4f}")

# --- Pozostałe ---
print("\n--- Pozostałe ---")
print(f"Specificity (avg): {best_trial.user_attrs['specificity_avg']:.4f} ± {best_trial.user_attrs['specificity_std']:.4f}")
print(f"MSE: {best_trial.user_attrs['mse']:.4f} ± {best_trial.user_attrs['mse_std']:.4f}")
print(f"RMSE: {best_trial.user_attrs['rmse']:.4f} ± {best_trial.user_attrs['rmse_std']:.4f}")
print(f"LogLoss: {best_trial.user_attrs['logloss']:.4f} ± {best_trial.user_attrs['logloss_std']:.4f}")

print("\n🧮 Confusion Matrix:")
print(np.array(best_trial.user_attrs["confusion_matrix"]))



📊 Metryki najlepszej próby:
--- Mikro ---
F1-score (micro): 0.8225 ± 0.0402
Accuracy: 0.8225 ± 0.0402
Precision (micro): 0.8225 ± 0.0402
Recall / Sensitivity (micro): 0.8225 ± 0.0402

--- Makro ---
F1-score (macro): 0.7728 ± 0.0415
Precision (macro): 0.8020 ± 0.0482
Recall / Sensitivity (macro): 0.7534 ± 0.0374

--- Weighted ---
F1-score (weighted): 0.8219 ± 0.0401
Precision (weighted): 0.8250 ± 0.0396
Recall / Sensitivity (weighted): 0.8225 ± 0.0402

--- Pozostałe ---
Specificity (avg): 0.9292 ± 0.0166
MSE: 0.2238 ± 0.0534
RMSE: 0.4694 ± 0.0591
LogLoss: 0.5116 ± 0.0477

🧮 Confusion Matrix:
[[505  62   2   1]
 [ 57 325  18   2]
 [  4  35  71   5]
 [  0   7  10  40]]


In [ ]:
# Wyświetlenie etykiet klas w kolejności odpowiadającej macierzy pomyłek
class_labels = le.classes_
print("Klasy w macierzy pomyłek:", class_labels)

# Macierz pomyłek zsumowana po 5 foldach
conf_mat_global = np.array(best_trial.user_attrs["confusion_matrix"])
print("Globalna macierz pomyłek:\n", conf_mat_global)

Klasy w macierzy pomyłek: ['-03:00:00' '-04:00:00' '-05:00:00' '-06:00:00' '-08:00:00' '01:00:00'
 '02:00:00' '03:00:00' '04:00:00' '08:00:00' '09:00:00' '10:00:00'
 '11:00:00']
Globalna macierz pomyłek:
 [[509  55   5   1]
 [ 63 315  20   4]
 [  5  45  60   5]
 [  1  10   8  38]]


In [ ]:
!pip freeze > requirements.txt

In [ ]:
# # Definicja klasyfikatorów
# svm_clf = SVC(probability=True, kernel='rbf', random_state=42)
# rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)
# gb_clf = GradientBoostingClassifier(n_estimators=100, random_state=42)

In [ ]:
# Model zespołowy VotingClassifier
# ensemble_clf = VotingClassifier(
#     estimators=[('svm', svm_clf), ('rf', rf_clf), ('gb', gb_clf)],
#     voting='soft'
# )

In [ ]:
# ensemble_clf.fit(X_train, y_train)

In [ ]:
# y_pred = ensemble_clf.predict(X_test)

In [ ]:
# # Predykcje klasowe i probabilistyczne
# y_pred = ensemble_clf.predict(X_test)
# y_proba = ensemble_clf.predict_proba(X_test)  # potrzebne do LogLoss

In [ ]:

# # Macierz pomyłek
# cm = confusion_matrix(y_test, y_pred)
# print("Macierz pomyłek:\n", cm)

In [ ]:
# # Obliczamy metryki na poziomie makro (średnia po klasach)
# accuracy = accuracy_score(y_test, y_pred)
# precision = precision_score(y_test, y_pred, average='macro')
# recall = recall_score(y_test, y_pred, average='macro')
# f1 = f1_score(y_test, y_pred, average='macro')

# print(f"Accuracy: {accuracy:.4f}")
# print(f"Precision (macro): {precision:.4f}")
# print(f"Recall / Sensitivity (macro): {recall:.4f}")
# print(f"F1-score (macro): {f1:.4f}")

In [ ]:
# specificity_per_class = []
# for i in range(len(cm)):
#     TP = cm[i, i]
#     FP = cm[:, i].sum() - TP
#     FN = cm[i, :].sum() - TP
#     TN = cm.sum() - (TP + FP + FN)
#     specificity = TN / (TN + FP) if (TN + FP) > 0 else 0
#     specificity_per_class.append(specificity)

# print("Specificity per class:", specificity_per_class)
# print(f"Average specificity: {np.mean(specificity_per_class):.4f}")

In [ ]:
# # Konwersja etykiet na numeryczne, jeśli trzeba
# from sklearn.preprocessing import LabelEncoder
# le = LabelEncoder()
# y_test_num = le.fit_transform(y_test)
# y_pred_num = le.transform(y_pred)

# mse = mean_squared_error(y_test_num, y_pred_num)
# rmse = np.sqrt(mse)

# print(f"MSE: {mse:.4f}")
# print(f"RMSE: {rmse:.4f}")

In [ ]:
# # LogLoss (wymaga y_test w formie numerycznej oraz prawdopodobieństw z modelu)
# logloss = log_loss(y_test_num, y_proba)
# print(f"LogLoss: {logloss:.4f}")